In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [59]:
import os

file_path = './mnist_test.csv'  # Update if needed
print("File exists:", os.path.exists(file_path))

data = pd.read_csv(file_path)

File exists: True


In [60]:
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
data.shape

(10000, 785)

In [62]:
x_traning = data.iloc[0:8000, 1:].to_numpy()
y_traning = data.iloc[0:8000, :1].to_numpy()

In [63]:
print(x_traning.shape)
y_traning.shape

(8000, 784)


(8000, 1)

In [64]:
print(x_traning)
print(y_traning)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[7]
 [2]
 [1]
 ...
 [5]
 [7]
 [8]]


In [ ]:
x_test = data.iloc[8000:, 1:].to_numpy()
y_test = data.iloc[8000:, :1].to_numpy()

In [ ]:
def init_weight_bias(x, y):
    ''' 
    Initialize weights and bias for a neural network
    '''
    
    x_rows, x_cols = x.shape
    y_rows, y_cols = y.shape
    w = np.random.rand(x_cols, x_rows)
    b = np.random.rand(y_rows, 1)  # Ensures bias matches number of outputs
    return w, b

In [67]:
w, b = init_weight_bias(x_traning, y_traning)
print(w.shape)
print(b.shape)

(784, 8000)
(8000, 1)
